# Introduction

Hi Leo, this is very similar to what we need to do:

We will now formulate the diet problem as an optimization model. As discussed in class, there are three components:
1. **Decision variables:** We create a decision variable $x_f$ for the quantity picked from each food item $f \in {\rm foods}$, meaning that we have the following decision variables: $x_{chicken}, x_{greens}, x_{salad},x_{bread}$. The quantity in the optimal diet is unknown; $x_f$ could potentially take any non-negative value.  The solver will find an optimal value $x^*_f$ for each food item $f$. For example, if the optimal diet does not contain any "bread", we will obtain $x^*_{bread} = 0$.


2. **Constraints:** We need to guarantee a minimal daily intake for each nutrient. These requirements form an ensemble of *constraints*. For example, we need to ensure that at least 1800 calories are provided by the meal. Mathematically, this requirement is expressed using a linear constraint: 
<br><br>
$$ 420 \cdot x_{chicken} + 34\cdot x_{greens} + 50\cdot x_{salad} + 330\cdot x_{bread} \geq 1800$$
<br>
In this constraint, we multiply the quantity of each food item by the corresponding calorie contribution per unit. The total contribution should exceed the requirement of 1800. We will need one constraint for each nutrient: calories, protein and sodium.
    
    
3. **Objective function:** Lastly, we need to select the objective function. *What should we optimize for?* The goal is find a menu composition that minimizes the total cost. The cost is given by the following expression:
<br><br>
$$ 1.69 \cdot x_{chicken} + 0.49\cdot x_{greens} + 0.49\cdot x_{salad} + 0.99\cdot x_{bread}$$
<br>
We multiply the quantity of each food item by the corresponding cost per unit. The objective function is obtained by taking the sum.

We now write down the optimization model:

$$ \max_{x_f} \quad\quad\quad  1.69 \cdot x_{chicken} + 0.49\cdot x_{greens} + 0.49\cdot x_{salad} + 0.99\cdot x_{bread}$$
$\quad\quad{\rm subject \ to}$
$$x_{chicken}, x_{greens}, x_{salad},x_{bread} \geq 0$$
$$ \quad  420 \cdot x_{chicken} + 34\cdot x_{greens} + 50\cdot x_{salad} + 330\cdot x_{bread} \geq 1800 $$ 
$$ \quad  32 \cdot x_{chicken} + 15\cdot x_{greens} + 3\cdot x_{salad} + 7\cdot x_{bread} \geq 200 $$ 
$$ \quad  1190 \cdot x_{chicken} + 33\cdot x_{greens} + 420\cdot x_{salad} + 100\cdot x_{bread} \geq 0 $$ 

This form of optimization models is referred to as a *linear program*. We often use the abbrevation *LP* to designate a linear program. This is because the constraints and the objective function are affine functions of the decision variables $x_f$.

## Load the data

To find the optimal purchasing decisions (minimal cost) for a consumer trying to reach their recommended caloric intake, we have data on:
- Daily intake for different categories of macronutrients
- Costs of various food items

In [32]:
# Import various packages
import pandas as pd
import numpy as np
import seaborn as sns # general visualization package  
import matplotlib.pyplot as plt # general visualization package 
from collections import defaultdict #needed to make 
#next command allows you to display the figures in the notebook  
%matplotlib inline   

In [22]:
#List of nutrient categories
categories = ['Calories', 'Fat','Saturates', 'Carbohydrates', 'Sugar', 'Fibre', 'Protein', 'Salt']

#initially only care about the calories
minNutrition = {
                'Calories': 1800,
                'Fat': 0,
                'Saturates': 0,
                'Carbohydrates':0,
                'Sugar': 0,
                'Fibre': 0,
                'Protein': 0,
                'Salt':0,
                }

In [27]:
meal_df = pd.read_csv("DAM_Meal_Dataset.csv") 
meal_df.columns

Index(['Supermarket', 'Meal', 'Price', 'Review', 'Calories', 'Fat',
       'Saturates', 'Carbohydrates', 'Sugar', 'Fibre', 'Protein', 'Salt',
       'Cooking Time', 'Vegan'],
      dtype='object')


Using the command `.describe()`, you can familiarize yourself with the format of the dataframes, and  check if there are missing entries.

In [28]:
meal_df.head()

,Supermarket,Meal,Price,Review,Calories,Fat,Saturates,Carbohydrates,Sugar,Fibre,Protein,Salt,Cooking Time,Vegan
0,Tesco,Tesco Finest Cottage Pie 400G,3.75,2.4,418.0,13.9,6.7,43.4,5.8,5.5,27.1,1.90,35,0
1,Tesco,Charlie Bigham's Lasagne 355G,5.00,4.9,713.0,41.7,19.8,37.9,10.6,NaN,40.2,2.42,30,0
2,Tesco,Tesco Chicken Jalfrezi & Pilau Rice 450G,2.75,3.7,446.0,10.1,1.7,51.5,9.7,12.7,30.8,1.90,15,0
3,Tesco,Tesco Finest Chicken Korma With Pilau Rice 450G,3.75,1.8,556.0,21.0,11.5,59.8,8.3,8.3,27.7,2.50,30,0
4,Tesco,Tesco Finest Roasted Harissa Squash With Orzo ...,3.75,2.9,384.0,11.9,1.0,54.0,18.7,10.4,10.0,1.76,25,1


In [17]:
meals =meal_df["Meal"].tolist()
prices = meal_df["Price"].tolist()

In [31]:
zip_iterator = zip(meals, prices)
# This will create a dictionary which has 
cost = dict(zip_iterator)
# {'Tesco Finest Cottage Pie 400G': 3.75
#  'Charlie Bigham's Lasagne 355G':	5.00
# } ....

In [51]:
meal_df.loc[meal_df['Meal'] == 'Tesco Finest Cottage Pie 400G', 'Fat'].values[0]

13.9

In [56]:
# WE ARE TRYING TO CREATE THIS:
# [('Tesco Finest Cottage Pie 400G', 'Calories'): 380,
#  ('Tesco Finest Cottage Pie 400G', 'Fat'): 22, 
#  ('Tesco Finest Cottage Pie 400G', 'Saturates'), ....

meal_cats = []
values = []
for x in meals:
    for y in categories:
        meal_cats.append((x,y))
        temp = meal_df.loc[meal_df['Meal'] == x, y].values[0] # finds the value of the meal and category
        values.append(temp)
        
        

zip_iterator2 = zip(meal_cats, values)
# This will create a dictionary which has 
nutritionValues = dict(zip_iterator2)
# nutritionValues


{('Tesco Finest Cottage Pie 400G', 'Calories'): 418.0,
 ('Tesco Finest Cottage Pie 400G', 'Fat'): 13.9,
 ('Tesco Finest Cottage Pie 400G', 'Saturates'): 6.7,
 ('Tesco Finest Cottage Pie 400G', 'Carbohydrates'): 43.4,
 ('Tesco Finest Cottage Pie 400G', 'Sugar'): 5.8,
 ('Tesco Finest Cottage Pie 400G', 'Fibre'): 5.5,
 ('Tesco Finest Cottage Pie 400G', 'Protein'): 27.1,
 ('Tesco Finest Cottage Pie 400G', 'Salt'): 1.9,
 ("Charlie Bigham's Lasagne 355G", 'Calories'): 713.0,
 ("Charlie Bigham's Lasagne 355G", 'Fat'): 41.7,
 ("Charlie Bigham's Lasagne 355G", 'Saturates'): 19.8,
 ("Charlie Bigham's Lasagne 355G", 'Carbohydrates'): 37.9,
 ("Charlie Bigham's Lasagne 355G", 'Sugar'): 10.6,
 ("Charlie Bigham's Lasagne 355G", 'Fibre'): 6.2,
 ("Charlie Bigham's Lasagne 355G", 'Protein'): 40.2,
 ("Charlie Bigham's Lasagne 355G", 'Salt'): 2.42,
 ('Tesco Chicken Jalfrezi & Pilau Rice 450G', 'Calories'): 446.0,
 ('Tesco Chicken Jalfrezi & Pilau Rice 450G', 'Fat'): 10.1,
 ('Tesco Chicken Jalfrezi & Pilau

In [55]:
len(values)
len(tuples)

288

In [5]:
# The following lines of code import the gurobi package
import gurobipy as gp
from gurobipy import GRB,quicksum

## Model creation

In [7]:
#Insert your code here:
m=gp.Model('diet_choice')

## Decision variables

## Constraints

Cannot choose the same meal more than 3 times in a week.

Need X amount of protein.



In [10]:
#Insert your code here:
m.addConstrs()
             
             
             

{119: <gurobi.Constr *Awaiting Model Update*>,
 120: <gurobi.Constr *Awaiting Model Update*>,
 127: <gurobi.Constr *Awaiting Model Update*>,
 143: <gurobi.Constr *Awaiting Model Update*>,
 144: <gurobi.Constr *Awaiting Model Update*>,
 146: <gurobi.Constr *Awaiting Model Update*>,
 150: <gurobi.Constr *Awaiting Model Update*>,
 157: <gurobi.Constr *Awaiting Model Update*>,
 161: <gurobi.Constr *Awaiting Model Update*>,
 167: <gurobi.Constr *Awaiting Model Update*>,
 173: <gurobi.Constr *Awaiting Model Update*>,
 174: <gurobi.Constr *Awaiting Model Update*>,
 195: <gurobi.Constr *Awaiting Model Update*>,
 2000: <gurobi.Constr *Awaiting Model Update*>,
 2002: <gurobi.Constr *Awaiting Model Update*>,
 2003: <gurobi.Constr *Awaiting Model Update*>,
 2005: <gurobi.Constr *Awaiting Model Update*>,
 2009: <gurobi.Constr *Awaiting Model Update*>,
 2012: <gurobi.Constr *Awaiting Model Update*>,
 2017: <gurobi.Constr *Awaiting Model Update*>,
 2021: <gurobi.Constr *Awaiting Model Update*>,
 2022

## Objective

### Q2.7. How would you formulate the net revenue as a linear expression? Specify the objective of the model.

*Hint: Recall that the objective function is specified using:* `m.setObjective(EXPRESSION,GRB.MAXIMIZE)`

*Insert your answer here:* 


In [13]:

m.setObjective(quicksum()

## Solve

In [14]:
# Run the optimization
# Note: it is not convenient to printout the relocation solution. We will develop a suitable visualization tool.
def printSolution():
    if m.status == GRB.OPTIMAL:
        print('\nNet revenue: %g' % m.objVal)
    else:
        print('No solution:', m.status)
        
m.optimize()
printSolution()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[x86])
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 2067 rows, 475410 columns and 1424163 nonzeros
Model fingerprint: 0x705b3843
Variable types: 0 continuous, 475410 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-04, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 7e+02]
Found heuristic solution: objective 62468.000000
Presolve removed 689 rows and 19978 columns
Presolve time: 4.18s
Presolved: 1378 rows, 455432 columns, 1337358 nonzeros
Variable types: 0 continuous, 455432 integer (6671 binary)
Found heuristic solution: objective 62739.189353

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.5668248e+04   6.256250e+02   0.000000e+00      5s

Starting sifting (using dual simplex for sub-problems)...

    Iter     Pivots    Primal Obj      Dual Obj        Time


In [ ]:
# Insert your code in the cells below:
